In [1]:
#!import ./config/Settings.cs

bool useAzureOpenAI = true;

await Settings.AskAzureEndpoint(useAzureOpenAI);
await Settings.AskModel(useAzureOpenAI);
await Settings.AskApiKey(useAzureOpenAI);

Settings: OK: Azure OpenAI endpoint configured [./config/settings.json]
Settings: OK: deployment name configured [./config/settings.json]
Settings: OK: API key configured [./config/settings.json]


In [2]:
#!import ./config/Settings.cs

// Uncomment the line below to reset your settings and run step 1.1 again so it asks you for your API key
// Settings.Reset();

In [3]:
#r "nuget: Microsoft.SemanticKernel, 1.4.0"

#!import ./config/Settings.cs

using Microsoft.SemanticKernel;
using System.IO;

var kernelBuilder = Microsoft.SemanticKernel.Kernel.CreateBuilder();

// Grab the locally stored credentials from the settings.json file. 
// Name the service as "davinci" — assuming that you're using one of the davinci completion models. 
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    kernelBuilder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    kernelBuilder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = kernelBuilder.Build();

Installed Packages Microsoft.SemanticKernel, 1.4.0

In [4]:
string mySemanticFunctionInline = """
{{$input}}

Summarize the content above in less than 140 characters.
""";

Console.WriteLine("A string has been set to be used as a semantic function.");

A string has been set to be used as a semantic function.


In [5]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var executionSettings = new OpenAIPromptExecutionSettings
{
    MaxTokens = 2000,
    Temperature = 0.2,
    TopP = 0.5
};

var promptTemplateConfig = new PromptTemplateConfig(mySemanticFunctionInline);
var promptTemplateFactory = new KernelPromptTemplateFactory();
var promptTemplate = promptTemplateFactory.Create(promptTemplateConfig);

var renderedPrompt = await promptTemplate.RenderAsync(kernel);

Console.WriteLine(renderedPrompt);

var summaryFunction = kernel.CreateFunctionFromPrompt(mySemanticFunctionInline, executionSettings);

Console.WriteLine("A semantic function has been registered.");



Summarize the content above in less than 140 characters.
A semantic function has been registered.


In [6]:
var input = """
I think with some confidence I can say that 2023 is going to be the most exciting year that 
the AI community has ever had,” writes Kevin Scott, chief technology officer at Microsoft, 
in a Q&A on the company’s AI blog. He acknowledges that he also thought 2022 was the most 
exciting year for AI, but he believes that the pace of innovation is only increasing. 
This is particularly true with generative AI, which doesn’t simply analyze large data sets 
but is a tool people can use to create entirely new works. We can already see its promise 
in systems like GPT-3, which can do anything from helping copyedit and summarize text to 
providing inspiration, and DALL-E 2, which can create useful and arresting works of art 
based on text inputs. Here are some of Scott’s predictions about how AI will change the 
way we work and play.
""";
// Text source: https://www.microsoft.com/en-us/worklab/kevin-scott-on-5-ways-generative-ai-will-transform-work-in-2023

var summary = await kernel.InvokeAsync(summaryFunction, new() { ["input"] = input });

Console.WriteLine(summary);

Microsoft's CTO, Kevin Scott, predicts 2023 will be an exciting year for AI, especially generative AI like GPT-3 and DALL-E 2, transforming work and play.

